<a href="https://colab.research.google.com/github/kywch/geo-colab/blob/master/Generate_Chicago_Traces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install rasterio

     |████████████████████████████████| 18.1MB 1.5MB/s 


In [39]:
# Import necessary packages
import os, os.path
from glob import glob
from shutil import copyfile
import pickle

import pandas as pd
import numpy as np

from secrets import choice

import matplotlib.pyplot as plt
from scipy.interpolate import griddata

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Loading SynthPop data

These files contain people's simulated movements across 24 hr. 

In [5]:
files = glob("drive/My Drive/SatTemp/chicago_synthpop_V2/*.csv.gz")
print(files)

['drive/My Drive/SatTemp/chicago_synthpop_V2/persons_v2.csv.gz', 'drive/My Drive/SatTemp/chicago_synthpop_V2/places_v2_256p_lb_v2.csv.gz', 'drive/My Drive/SatTemp/chicago_synthpop_V2/activities_v2.csv.gz', 'drive/My Drive/SatTemp/chicago_synthpop_V2/places_v2.csv.gz', 'drive/My Drive/SatTemp/chicago_synthpop_V2/places_60615.csv.gz', 'drive/My Drive/SatTemp/chicago_synthpop_V2/persons_60615.csv.gz', 'drive/My Drive/SatTemp/chicago_synthpop_V2/sampled_persons.csv.gz']


In [6]:
!rm -rf synthpop
if not os.path.exists('synthpop'):
  os.mkdir('synthpop')

In [7]:
# copy these files from Google Drive to colab disk
for f in files:
  print(f)
  copyfile(f, 'synthpop/' + os.path.basename(f))

drive/My Drive/SatTemp/chicago_synthpop_V2/persons_v2.csv.gz
drive/My Drive/SatTemp/chicago_synthpop_V2/places_v2_256p_lb_v2.csv.gz
drive/My Drive/SatTemp/chicago_synthpop_V2/activities_v2.csv.gz
drive/My Drive/SatTemp/chicago_synthpop_V2/places_v2.csv.gz
drive/My Drive/SatTemp/chicago_synthpop_V2/places_60615.csv.gz
drive/My Drive/SatTemp/chicago_synthpop_V2/persons_60615.csv.gz
drive/My Drive/SatTemp/chicago_synthpop_V2/sampled_persons.csv.gz


In [8]:
# look at people
persons_df = pd.read_csv('synthpop/persons_v2.csv.gz')
print(len(persons_df))
#persons_df.info()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


2927761


In [9]:
# places
places_df = pd.read_csv('synthpop/places_v2.csv.gz')
places_df['numeric_id'] = places_df['numeric_id'].astype('int64')
#places_df.info()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
# make it faster to look-up
plidx_df = places_df.set_index('numeric_id')
#plidx_df.info()

In [11]:
# activities
activity_df = pd.read_csv('synthpop/activities_v2.csv.gz')
#activity_df.info()

In [12]:
# check the number of unique persons
#print(len(persons_df.person_id), len(persons_df.person_id.unique()))
#persons_df.head()

# Things to do
* join zip code to person -- sample 1000 each?
* for each person
  * grab the activity schedule
  * link place coordinates
  * make a row for each hour: 24 rows with coordinates
  * the coordinates for 24 hours will be used to query temperature (or heat stress)


  

In [13]:
person_row = next(persons_df.iterrows())[1]
print(person_row)

numeric_id                                                                  1
person_id                                                             1595930
hh_id                                                             1.18152e+06
relate                                                                      1
sex                                                                         2
age                                                                        82
gq_id                                                                     NaN
school_id                                                                 NaN
work_id                                                                   NaN
daycare_id                                                             579139
gym_id                                                            1.18126e+06
hospital_id                                                                70
jail                                                            

In [14]:
# %timeit places_df[places_df['numeric_id'] == 1210303]
# %timeit places_df.query('numeric_id == 1210303')
# %timeit plidx_df.loc[1210303]

In [15]:
# %timeit plidx_df.loc[[1197516,1200938,1195562,1196781,1195562]]
# %timeit [plidx_df.loc[1197516], plidx_df.loc[1200938], plidx_df.loc[1195562], plidx_df.loc[1196781], plidx_df.loc[1195562]]
# %timeit plidx_df.iat[1197517, 4]

In [16]:
def match_activity_with_coord(person_row, activity_df, sch_type='weekday'):

  # retrieve a schedule
  curr_activity = activity_df[activity_df['schedule_id'] == int(choice(person_row[sch_type+'_schedule_id_list'].split('|')))].reset_index(drop=True)

  place_list = curr_activity['Social_Act_Loc'].to_list()
  place_info = []
  prev_place = ''

  item_to_key = {
    'Household': 'hh_id', 
    'Work Restaurant': 'work_restaurant_id',
    'Other': 'hh_id', 
    'Other Household': 'otherhh_id',
    'Place of worship': 'worship_id', 
    'Home Restaurant': 'home_restaurant_id', 
    'Recreation': 'recreation_id',
    'Grocery store': 'grocery_id', 
    'School': 'school_id', 
    'Workplace': 'work_id', 
    'Gym': 'gym_id', 
    'Prison': 'jail_id'
  }

  for item in place_list:
    if item == prev_place:
      curr_info = place_info[-1]
    else:
      if item in item_to_key:
        try:
          if type(person_row[item_to_key[item]]) is str:
            idx = int(choice(person_row[item_to_key[item]].split('|')))
          else:
            idx = int(person_row[item_to_key[item]])
          curr_info = [item, idx, plidx_df.iat[idx, 4], plidx_df.iat[idx, 5]]
        except:
          # if no place was found, force assign household
          idx = int(person_row['hh_id'])
          curr_info = ['FORCE_HH', idx, plidx_df.iat[idx, 4], plidx_df.iat[idx, 5]]
      else:
        curr_info = ['FORCE_HH', idx, plidx_df.iat[idx, 4], plidx_df.iat[idx, 5]]

    #print(curr_info)
    place_info.append(curr_info)
    prev_place = item

  # return the horizontally-concated dataframe
  return(pd.concat([curr_activity, 
                    pd.DataFrame(place_info, columns=['item', 'place_numeric', 'lat', 'lng'])], 
                    axis=1))

%timeit match_activity_with_coord(person_row, activity_df)
#print(place_info)  

The slowest run took 43.63 times longer than the fastest. This could mean that an intermediate result is being cached.
100 loops, best of 3: 2.88 ms per loop


In [17]:
# hour, weight, social_act_loc, coordinates (lat/lng)
# using iat (instead of iloc) give ~10x speed improvement

def generate_hourly_activity(act_df):

  schedule = []
  for ii in range(act_df.shape[0]):
    curr_hour = act_df.iat[ii,1] // 60 #.iloc[ii].start_time // 60 
    proc_time = act_df.iat[ii,1] #.start_time

    # take care of the short activity
    if 60*(curr_hour+1) < act_df.iat[ii,2]: #iloc[ii].stop_time:
        schedule.append([curr_hour, 
                        (60*(curr_hour+1) - act_df.iat[ii,1])/60, 
                        act_df.iat[ii,3], act_df.iat[ii,7], act_df.iat[ii,8]]) #.iloc[ii].Social_Act_Loc])
        curr_hour += 1
        proc_time = 60*curr_hour

    # take care of the remaining long activity
    while proc_time < act_df.iat[ii,2]: #iloc[ii].stop_time:
      if (proc_time + 60) < act_df.iat[ii,2]: #iloc[ii].stop_time:
        schedule.append([curr_hour, 1, act_df.iat[ii,3], act_df.iat[ii,7], act_df.iat[ii,8]])
        proc_time += 60
        curr_hour += 1
      else:
        res_act = act_df.iat[ii,2] - proc_time
        schedule.append([curr_hour, res_act/60, act_df.iat[ii,3], act_df.iat[ii,7], act_df.iat[ii,8]])
        proc_time = act_df.iat[ii,2]
    
  # make the schedule compact
  compact_schedule = []
  compact_schedule.append(schedule.copy()[0])
  for item in schedule[1:]:
    # if the hour and activity are the same, just merge the time
    if (item[0] == compact_schedule[-1][0]) & (item[2] == compact_schedule[-1][2]):
      compact_schedule[-1][1] += item[1]
    else:
      compact_schedule.append(item)  

  # done
  return compact_schedule


In [18]:
row = persons_df.sample().iloc[0]
%timeit match_activity_with_coord(row, activity_df)

test = match_activity_with_coord(row, activity_df)
%timeit generate_hourly_activity(test)

100 loops, best of 3: 2.94 ms per loop
100 loops, best of 3: 7.2 ms per loop


## join zip code to person -- sample 1000 each?

In [25]:
generate_new_sample = False
num_sample_zip = 100

if generate_new_sample:
  
  # link people and zipcode
  print('Linking people and zipcode')
  pp_loc_df = persons_df.merge(plidx_df['zipcode'], left_on='hh_id', right_index=True)
  pp_loc_df.info()

  # get zipcode that has more than 1000 people in it
  tmp_zip = pp_loc_df.zipcode.value_counts()
  zipcode = list(tmp_zip[tmp_zip > 1000].index)
  print('The number of zip codes to sample: ', len(zipcode))

  # sampling 
  print('Sampling ', num_sample_zip, ' from each zip code')
  sample = pd.DataFrame()
  for zip in zipcode:
    sample = sample.append(pp_loc_df[pp_loc_df['zipcode'] == zip].sample(num_sample_zip))
  
  # save the sample
  sample.to_csv('drive/My Drive/SatTemp/chicago_synthpop_V2/sampled_persons.csv.gz')

else:
  sample = pd.read_csv('synthpop/sampled_persons.csv.gz')

len(sample)

5300

In [23]:
#sample.info()

In [24]:
def test():
  for index, person_row in sample.iterrows():
    generate_hourly_activity(
        match_activity_with_coord(person_row, activity_df)
    )

# %timeit test()

# Load the raster data

In [26]:
copyfile("drive/My Drive/SatTemp/SatTemp_Glynn_SELECT/processed.pkl", 'processed_raster.pkl')

'processed_raster.pkl'

In [28]:
# !ls

In [29]:
with open('processed_raster.pkl', 'rb') as file:
  geo_data = pickle.load(file)

geo_data.keys()

# using the same name
masked_raster = geo_data['lst_raster']
raster_shape = masked_raster[0].shape 
raster_bound = geo_data['boundary']['l00']

In [30]:
# convert lat/lng to raster xx, yy

uchicago_loc = [41.7886, -87.5987] # lat, lng

def coord_to_raster(coord, bound, shape, verbose=False):
  slope_lat = (bound[3] - bound[1]) / shape[0]
  slope_lng = (bound[2] - bound[0]) / shape[1]
  rst_lat = int(round((bound[3] - coord[0]) / slope_lat)) # lat
  rst_lng = int(round((coord[1] - bound[0]) / slope_lng)) # lng
  if verbose:
    print(coord, ' : (', rst_lat, ',', rst_lng, ')')
  return (rst_lat, rst_lng)

coord_to_raster([raster_bound[1],raster_bound[0]], raster_bound, raster_shape, verbose=True)
coord_to_raster([raster_bound[1],raster_bound[2]], raster_bound, raster_shape, verbose=True)
coord_to_raster([raster_bound[3],raster_bound[0]], raster_bound, raster_shape, verbose=True)
coord_to_raster([raster_bound[3],raster_bound[2]], raster_bound, raster_shape, verbose=True)
coord_to_raster(uchicago_loc, raster_bound, raster_shape, verbose=True)

%timeit coord_to_raster(uchicago_loc, raster_bound, raster_shape)

[41.46612652341049, -88.26368408376308]  : ( 685 , 0 )
[41.46612652341049, -87.31312550993508]  : ( 685 , 952 )
[42.14968408376307, -88.26368408376308]  : ( 0 , 0 )
[42.14968408376307, -87.31312550993508]  : ( 0 , 952 )
[41.7886, -87.5987]  : ( 362 , 666 )
The slowest run took 8.07 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 1.14 µs per loop


In [31]:
# convert temperature to heat stress
# no stress below 20 degress, linearly-increasing stress above 20

crit_temp = 20 # degress C

stress_raster = np.where((masked_raster-crit_temp) < 0, 0, (masked_raster-crit_temp))

stress_raster.shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in less
  


(24, 685, 952)

In [32]:
person_row = sample.iloc[0]
tmp = generate_hourly_activity(
        match_activity_with_coord(person_row, activity_df)
      )
print(tmp)

[[0, 1.0, 'Household', 41.7819731103871, -87.73442549545759], [1, 1, 'Household', 41.7819731103871, -87.73442549545759], [2, 1, 'Household', 41.7819731103871, -87.73442549545759], [3, 1, 'Household', 41.7819731103871, -87.73442549545759], [4, 1, 'Household', 41.7819731103871, -87.73442549545759], [5, 1, 'Household', 41.7819731103871, -87.73442549545759], [6, 1.0, 'Household', 41.7819731103871, -87.73442549545759], [7, 1.0, 'Workplace', 41.844707, -87.671752], [8, 1.0, 'Workplace', 41.844707, -87.671752], [9, 1, 'Workplace', 41.844707, -87.671752], [10, 1, 'Workplace', 41.844707, -87.671752], [11, 1.0, 'Workplace', 41.844707, -87.671752], [12, 1.0, 'Workplace', 41.844707, -87.671752], [13, 1.0, 'Workplace', 41.844707, -87.671752], [14, 1, 'Workplace', 41.844707, -87.671752], [15, 1, 'Workplace', 41.844707, -87.671752], [16, 1.0, 'Workplace', 41.844707, -87.671752], [17, 0.5, 'Workplace', 41.844707, -87.671752], [17, 0.5, 'Other', 41.7819731103871, -87.73442549545759], [18, 0.58333333333

In [33]:
def get_stress(row, stress_raster, raster_bound, raster_shape):
  (xx,yy) = coord_to_raster(row[3:5], raster_bound, raster_shape)
  try:
    return stress_raster[row[0], xx, yy] * row[1]
  except:
    return np.nan

print(get_stress(tmp[0], stress_raster, raster_bound, raster_shape))

%timeit get_stress(tmp[0], stress_raster, raster_bound, raster_shape)

0.42862081643829697
The slowest run took 6.33 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 5.27 µs per loop


In [71]:
#%timeit [get_stress(row, stress_raster, raster_bound, raster_shape) for row in tmp]

def get_stress_person(person_row, activity_df, stress_raster, raster_bound, raster_shape):
  #print(person_row)  
  def nan_helper(vct):
    return np.isnan(vct), lambda z: z.nonzero()[0]

  stress_vct = [ 
                get_stress(row, stress_raster, raster_bound, raster_shape) 
                for row in generate_hourly_activity(
                    match_activity_with_coord(person_row, activity_df)
                )]
  
  check, xx = nan_helper(stress_vct)
  if np.isnan(sum(check)):
    stress_vct[check] = np.interp(xx(check), xx(~check), stress_vct(~check))

  return sum(stress_vct)

%timeit get_stress_person(person_row, activity_df, stress_raster, raster_bound, raster_shape)

100 loops, best of 3: 8.04 ms per loop


In [73]:
def calculate_stress_score(sample, verbose=False):
  
  stress_score = []

  for index, person_row in sample.iterrows():
    if verbose & (index % 300 == 299):
      print('Processing ', index)
      
    try:
      stress_score.append(
          get_stress_person(person_row, activity_df, stress_raster, raster_bound, raster_shape)
      )
    except:
      stress_score.append(np.nan)
  
  if verbose:
    print('Done.')
  return stress_score

#%timeit calculate_stress_score(sample)

stress_score = calculate_stress_score(sample, verbose=True)
print('Number of NaN values: ', sum(np.isnan(stress_score)))

Processing  299
Processing  599
Processing  899
Processing  1199
Processing  1499
Processing  1799
Processing  2099
Processing  2399
Processing  2699
Processing  2999
Processing  3299
Processing  3599
Processing  3899
Processing  4199
Processing  4499
Processing  4799
Processing  5099
Done.


In [80]:
stress_df = sample[['person_id', 'sex', 'age', 'zipcode']].merge(
    pd.DataFrame(stress_score, columns=['stress_score']), 
    left_index=True, right_index=True)

In [101]:
agg_by_zip = stress_df.groupby('zipcode').agg({
    'sex': lambda x: (x==1).mean(), 
    'age': np.nanmean, 
    'stress_score': [np.nanmean, lambda x: x.isnull().mean()]
}).reset_index('zipcode')

agg_by_zip.head()

zipcode      sex     age stress_score           
           <lambda> nanmean      nanmean <lambda_0>
0  60601.0     0.42   44.23     5.634215       0.00
1  60605.0     0.39   38.74    17.769965       0.00
2  60606.0     0.60   35.18    47.209015       0.01
3  60607.0     0.47   36.12    87.737735       0.01
4  60608.0     0.46   25.81    98.960949       0.00

In [103]:
agg_by_zip.columns = ['zipcode', 'gender_ratio', 'mean_age', 'stress_score', 'nan_ratio']

agg_by_zip.head()

,zipcode,gender_ratio,mean_age,stress_score,nan_ratio
0,60601.0,0.42,44.23,5.634215,0.00
1,60605.0,0.39,38.74,17.769965,0.00
2,60606.0,0.60,35.18,47.209015,0.01
3,60607.0,0.47,36.12,87.737735,0.01
4,60608.0,0.46,25.81,98.960949,0.00
